In [1]:
from __future__ import division
def feature_eng(y, fs, vis=False):
    
    import librosa
    import librosa.display
    import numpy as np
    import maad
    from soundsig.sound import BioSound
    import matplotlib.pyplot as plt
    import pandas as pd
    import warnings
    warnings.simplefilter('ignore')
    
    # spectrum
    ps = np.abs(np.fft.fft(y))**2
    time_step = 1/fs
    freqs = np.fft.fftfreq(y.size, time_step)
    ps = ps[0:int((len(ps)/2)-1)] # take out the negative freq
    freqs = freqs[0:int((len(freqs)/2)-1)]

    # melspectrogram
    S = librosa.feature.melspectrogram(y=y, sr=fs)
    S_dB = librosa.power_to_db(S, ref=np.max)
    
    # alpha indices
    Sxx_power,tn,fn,ext = maad.sound.spectrogram (y, fs, mode='psd')
    df_temporal_indices = maad.features.all_temporal_alpha_indices(y, fs)
    df_spectral_indices, _ = maad.features.all_spectral_alpha_indices(Sxx_power,tn,fn, extent=ext)
    df_indices = pd.concat([df_temporal_indices,df_spectral_indices], axis=1)
    
    # soundsig for modulation power spectrum
    
    myBioSound = BioSound(soundWave=y, fs=fs)
    myBioSound.mpsCalc(window=0.1, Norm = True)
    
    if vis==True:
        plt.figure()
        plt.plot(np.arange(0,y.size/fs,1/fs), y)
        plt.show()
        
        fig, ax = plt.subplots()
        img = librosa.display.specshow(S_dB, x_axis='time',
                                 y_axis='mel', sr=fs,
                                 fmax=fs/2, ax=ax)
        fig.colorbar(img, ax=ax, format='%+2.0f dB')
        ax.set(title='Mel-frequency spectrogram')
        
        DBNOISE=50
        plt.figure()
        plt.clf()
        cmap = plt.get_cmap('jet')
        ex = (myBioSound.wt.min(), myBioSound.wt.max(), myBioSound.wf.min()*1e3, myBioSound.wf.max()*1e3)
        logMPS = 10.0*np.log10(myBioSound.mps)
        maxMPS = logMPS.max()
        minMPS = maxMPS-DBNOISE
        logMPS[logMPS < minMPS] = minMPS
        plt.imshow(logMPS, interpolation='nearest', aspect='auto', origin='lower', cmap=cmap, extent=ex)
        plt.ylabel('Spectral Frequency (Cycles/KHz)')
        plt.xlabel('Temporal Frequency (Hz)')
        plt.colorbar()
        plt.ylim((0,myBioSound.wf.max()*1e3))
        plt.title('Modulation Power Spectrum')
        plt.show()
        
    
    return ps, S_dB, df_indices, myBioSound.mps, myBioSound.wt, myBioSound.wf
    

In [2]:
def preproc(file, fs=16000, vis=False):
    import librosa
    import numpy as np
    import matplotlib.pyplot as plt
    import noisereduce as nr
    
    raw_y, fs = librosa.load(file, sr=fs, duration=10, mono = True)
    y_mono_rs = raw_y - np.mean(raw_y) # remove DC
    rms = np.sqrt(np.mean(y_mono_rs**2)) # get rms
    y = y_mono_rs/(rms/0.1) # normalize the rms to 0.1
    
    fg_y = nr.reduce_noise(y=y, sr=fs)
    bg_y = y - fg_y
    
    
    if vis == True: print('++++++++++++++++++++++++ raw ++++++++++++++++++++++++')
    ps, S_dB, df_indices, mps, wt, wf = feature_eng(y, fs, vis)
    if vis == True: print('++++++++++++++++++++ foreground ++++++++++++++++++++')
    ps_fg, S_dB_fg, df_indices_fg, mps_fg, wt_fg, wf_fg = feature_eng(fg_y, fs, vis)
    if vis == True: print('++++++++++++++++++++ background ++++++++++++++++++++')
    ps_bg, S_dB_bg, df_indices_bg, mps_bg, wt_bg, wf_bg = feature_eng(bg_y, fs, vis)
    
    output = {'y': y, 'fg_y': fg_y, 'bg_y': bg_y, 'fs': fs,
             'ps': ps, 'S_dB': S_dB, 'df_indices': df_indices, 'mps': mps, 'wt': wt, 'wf': wf, 
             'ps_fg': ps_fg, 'S_dB_fg': S_dB_fg, 'df_indices_fg': df_indices_fg, 'mps_fg': mps_fg, 'wt_fg': wt_fg, 'wf_fg': wf_fg,
             'ps_bg': ps_bg, 'S_dB_bg': S_dB_bg, 'df_indices_bg': df_indices_bg, 'mps_bg': mps_bg, 'wt_bg': wt_bg, 'wf_bg': wf_bg}
    
    return output
    

In [3]:
# scan data directories
import glob
import pandas as pd

nature_file_list = []
nature_file_list += glob.glob('../data/raw/AmbisonicSoundLibrary/nature/*')
nature_file_list += glob.glob('../data/raw/GoogleAudioSet/Outside, rural or natural/*')
nature_file_list += glob.glob('../data/raw/youtube/NatureSoundscapes/*')
nature_file_list += glob.glob('../data/raw/youtube/NomadicAmbience_nature/*')
nature_file_list += glob.glob('../data/raw/S2L_LULC/non_urban/*')
nature_file_list += glob.glob('../data/raw/S2L_LULC/urban_0_25/*')

city_file_list = []
city_file_list += glob.glob('../data/raw/GoogleAudioSet/Outside, urban or manmade/*')
city_file_list += glob.glob('../data/raw/youtube/NomadicAmbience_city/*')
city_file_list += glob.glob('../data/raw/S2L_LULC/urban_26_100/*')

In [4]:
# as there are too many files in SONYC, so only a few are sampled
from random import seed, sample
seed(23)
SONYC_file_list = sample(glob.glob('../data/raw/SONYC/**/*.wav', recursive=True),800)
city_file_list += SONYC_file_list

pd.DataFrame({'nature_file_list': nature_file_list}).to_csv('../data/raw/nature_file_list.csv')
pd.DataFrame({'city_file_list': city_file_list}).to_csv('../data/raw/city_file_list.csv')

In [5]:
# testing cell
# preproc(city_file_list[0])

In [8]:
import time
import pickle
import os

from joblib import Parallel, delayed

def run_preproc(file_name):
    save_file_name = '../data/interim/'+file_name[12:-4]+'.pkl'
    if not os.path.isfile(save_file_name): # run the script only if the file does not exist
        if not os.path.exists(save_file_name.rsplit('/', 1)[0]): # create the folder if the folder does not exist
            os.makedirs(save_file_name.rsplit('/', 1)[0]) # extract the folder of the file path
            

        start_time = time.time()

        output = preproc(file_name, vis = False)
        f = open(save_file_name,'wb') # create a binary pickle file 
        pickle.dump(output,f)
        f.close()

        print("--- %s seconds ---" % (time.time() - start_time))
        

# run the process in parallel
Parallel(n_jobs=4)(delayed(run_preproc)(file_name) for file_name in nature_file_list+city_file_list)

--- 9.898964881896973 seconds ---
--- 9.100216150283813 seconds ---
--- 8.88844895362854 seconds ---
--- 7.146794080734253 seconds ---
--- 7.172313213348389 seconds ---
--- 9.33692193031311 seconds ---
--- 18.447308778762817 seconds ---
--- 10.943819999694824 seconds ---
--- 11.593420028686523 seconds ---
--- 9.468605279922485 seconds ---
--- 9.122115135192871 seconds ---
--- 8.108592748641968 seconds ---
--- 8.158255815505981 seconds ---
--- 9.75415325164795 seconds ---
--- 9.102648973464966 seconds ---
--- 8.638180017471313 seconds ---
--- 7.46058988571167 seconds ---
--- 7.1390509605407715 seconds ---
--- 8.069586277008057 seconds ---
--- 18.45258903503418 seconds ---
--- 11.116530179977417 seconds ---
--- 12.267534017562866 seconds ---
--- 9.959581136703491 seconds ---
--- 8.853277921676636 seconds ---
--- 8.029047012329102 seconds ---
--- 8.312987089157104 seconds ---
--- 11.731870889663696 seconds ---
--- 9.127806901931763 seconds ---
--- 8.369369268417358 seconds ---
--- 8.51460

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

--- 10.47667908668518 seconds ---
--- 6.355269193649292 seconds ---
--- 6.5009589195251465 seconds ---
--- 6.22754693031311 seconds ---
--- 6.168568134307861 seconds ---
--- 6.247599124908447 seconds ---
--- 6.493039131164551 seconds ---
--- 6.472807168960571 seconds ---
--- 6.158246755599976 seconds ---
--- 6.166565179824829 seconds ---
--- 6.212172985076904 seconds ---
--- 6.499074697494507 seconds ---
--- 5.889519929885864 seconds ---
--- 6.073751926422119 seconds ---
--- 6.099876880645752 seconds ---
--- 6.559180974960327 seconds ---
--- 6.432551860809326 seconds ---
--- 5.913939952850342 seconds ---
--- 6.243911981582642 seconds ---
--- 6.201107025146484 seconds ---
--- 6.35526704788208 seconds ---
--- 6.241070985794067 seconds ---
--- 6.096081972122192 seconds ---
--- 6.035611152648926 seconds ---
--- 5.951701879501343 seconds ---
--- 6.353864908218384 seconds ---
--- 6.240656852722168 seconds ---
--- 6.280117034912109 seconds ---
--- 6.219743251800537 seconds ---
--- 6.275204181

--- 10.128499984741211 seconds ---
--- 6.883440732955933 seconds ---
--- 6.799890041351318 seconds ---
--- 6.202651739120483 seconds ---
--- 6.394829034805298 seconds ---
--- 6.472589015960693 seconds ---
--- 6.847671985626221 seconds ---
--- 6.6936280727386475 seconds ---
--- 6.3579771518707275 seconds ---
--- 6.362604141235352 seconds ---
--- 6.364761829376221 seconds ---
--- 6.917219161987305 seconds ---
--- 6.712845087051392 seconds ---
--- 6.473498821258545 seconds ---
--- 6.580496788024902 seconds ---
--- 6.280734062194824 seconds ---
--- 6.615709066390991 seconds ---
--- 6.495178937911987 seconds ---
--- 6.5553669929504395 seconds ---
--- 6.628643035888672 seconds ---
--- 6.4490320682525635 seconds ---
--- 6.474807024002075 seconds ---
--- 6.26371693611145 seconds ---
--- 7.196225881576538 seconds ---
--- 6.821011066436768 seconds ---
--- 6.431702613830566 seconds ---
--- 6.11504602432251 seconds ---
--- 6.858770132064819 seconds ---
--- 9.854321241378784 seconds ---
--- 10.5753

In [ ]:
len(nature_file_list)+len(city_file_list)

In [7]:
for file_name in nature_file_list+city_file_list:
    print(file_name)
    run_preproc(file_name)

../data/raw/AmbisonicSoundLibrary/nature/Anna Bay Cicadas.mp3
../data/raw/AmbisonicSoundLibrary/nature/Wombat Forest Kookaburras.mp3
../data/raw/AmbisonicSoundLibrary/nature/Victoria Birds.mp3
../data/raw/AmbisonicSoundLibrary/nature/Cicadas at Dusk.mp3
../data/raw/AmbisonicSoundLibrary/nature/Torquay Surf.mp3
../data/raw/AmbisonicSoundLibrary/nature/Hurricane On Boat Deck.mp3
../data/raw/AmbisonicSoundLibrary/nature/Werribee Wetlands.mp3
../data/raw/AmbisonicSoundLibrary/nature/Daylesford Frogs.mp3
../data/raw/AmbisonicSoundLibrary/nature/Hepburn Blowhole.mp3
../data/raw/AmbisonicSoundLibrary/nature/Summer Rain Shower.mp3
../data/raw/AmbisonicSoundLibrary/nature/Guildford Cicadas.mp3
../data/raw/AmbisonicSoundLibrary/nature/Small Running Creek.mp3
../data/raw/AmbisonicSoundLibrary/nature/Waterfall Three Feet Above.mp3
../data/raw/AmbisonicSoundLibrary/nature/Strong Winds in a Forest.mp3
../data/raw/AmbisonicSoundLibrary/nature/Daylesford Spillway.mp3
../data/raw/AmbisonicSoundLibrary/

--- 7.734973192214966 seconds ---
../data/raw/S2L_LULC/non_urban/s2lam038_190318_2019-03-20_11-00.wav
--- 4.468991279602051 seconds ---
../data/raw/S2L_LULC/non_urban/s2lam045_190617_2019-06-17_18-20.wav
--- 4.728076219558716 seconds ---
../data/raw/S2L_LULC/non_urban/s2lam056_190528_2019-05-30_05-00.wav


KeyboardInterrupt: 